In [45]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tag import pos_tag
import feedparser
import urllib.parse
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
user_query = str(input('search a topic youd like news about:'))

search a topic youd like news about:trump ukraine executive order


In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/vjrdnti/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vjrdnti/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/vjrdnti/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
def process_query(query):
    tokenz = word_tokenize(query)
    sw = set(stopwords.words('english')) | {"news", "latest", "summary", "about", "give", "me"}
    tokens_no_stopwords = [word for word in tokenz if word.lower() not in sw]
    tagged_tokens = pos_tag(tokens_no_stopwords)
    noun_tokens = [word for word, tag in tagged_tokens if tag.startswith('NN')]
    noun_bigrams = []
    for i in range(len(noun_tokens)-1,0,-1):
        noun_bigrams.append( noun_tokens[i-1]+ ' ' + noun_tokens[i] )
    return [query, ' '.join(tokens_no_stopwords)] + noun_bigrams 

In [5]:
query_list = process_query(user_query)
query_list

['trump ukraine executive order',
 'trump ukraine executive order',
 'executive order']

In [6]:
def fetch_news(query):
    corrected_query = urllib.parse.quote(query)
    rss_url = f'https://news.google.com/rss/search?q={corrected_query}&hl=en-IN&gl=IN&ceid=IN:en'
    feed = feedparser.parse(rss_url)
    meta_data = []
    for entry in feed.entries:
        #print(entry)
        #break
        row = {
            'query': query,
            'title': entry.title,
            'link': entry.link,
            'published': entry.published,
            'published_parsed': datetime(*entry.published_parsed[:6])
        }
        meta_data.append(row)
    meta_data.sort(key=lambda x: x['published_parsed'], reverse=True)
    return meta_data[0:5]

In [13]:
news_results = []
for query in query_list[1:]:
    row = fetch_news(query)
    for r in row:
        if r['title'] not in news_results:
            news_results.append(r)
news_results

[{'query': 'trump ukraine executive order',
  'title': 'Trump raises alarm with Napoleon dictator quote as 20 immigration judges abruptly fired: Live updates - The Independent',
  'link': 'https://news.google.com/rss/articles/CBMi2AFBVV95cUxNem14cnE4M2REbjNIMEJCLVdJdkVxbFF5bXBLMWg5cFJDeW5HWlZMMjdoOEp2WThIWDAyUnlwVkVZbFZJcXVwdlVscUhoSDYxLU43OXFiTm1rREtoZ1JZcGppa1JQVlV4YU44MmZEWElCN1dUWFRwTGJWekhEdHptTDg1Y2F6NnVQY2pOZzFHUjNyUTkycm9IcEp0NUVuYlQ5NFBKVDBacHNSbmtCQWtzdmhrYVpTb0lEWk52ZXc3WkJqbk0tRlRoR1E4ZUlRUHRlZ1l0cmhQbG0?oc=5',
  'published': 'Sun, 16 Feb 2025 03:55:34 GMT',
  'published_parsed': datetime.datetime(2025, 2, 16, 3, 55, 34)},
 {'query': 'trump ukraine executive order',
  'title': 'Trump 2nd term live updates: Trump officials to head to Saudi Arabia for Ukraine peace talks, sources say - ABC News',
  'link': 'https://news.google.com/rss/articles/CBMilgFBVV95cUxNUGd1SXFmaERldmg3Tm1lV1lYLW1fOVR4U293NjVBRmktMl9MVXczc01aQ19YS1diZUVBNklEZkMzNHI3QlBTUVNIZmp3cThKN3Q4UHlDOXF6X1BobFFlcG

In [46]:
chrome_options = Options()
chrome_options.add_argument("--headless")  # no window :3
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

In [59]:
def scrape_news(news_list):
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(options=chrome_options)
    scraped_data = []
    for news in news_list:
        source = news['title'].split(' - ')[1]
        url = news["link"]
        if not url:
            print(f"Skipping {source}: no link")
            continue
        print(f"Visiting {source}: {url}")
        try:
            driver.get(url)
            time.sleep(2)
            final_url = driver.current_url
            print(f"Redirected to: {final_url}")
            driver.get(final_url)
            time.sleep(3)
            soup = BeautifulSoup(driver.page_source, "html.parser")
            paragraphs = [p.get_text(strip=True) for p in soup.find_all("p")]
            content = " ".join(paragraphs) if paragraphs else "No content found."

            scraped_data.append({"source": source, "content": content, "final_url": final_url, 'all': soup.text})

        except Exception as e:
            print(f"Error scraping {source}: {e}")

    driver.quit()
    return scraped_data

In [62]:
scraped_results = scrape_news(news_results[0:1]) #one element taaki jaldi ho testing

2025-02-16 16:29:52 [WDM] INFO: ====== WebDriver manager ======
2025-02-16 16:29:52 [WDM] INFO: Get LATEST chromedriver version for google-chrome
2025-02-16 16:29:52 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): googlechromelabs.github.io:443
2025-02-16 16:29:53 [urllib3.connectionpool] DEBUG: https://googlechromelabs.github.io:443 "GET /chrome-for-testing/latest-patch-versions-per-build.json HTTP/1.1" 200 9720
2025-02-16 16:29:53 [WDM] INFO: Get LATEST chromedriver version for google-chrome
2025-02-16 16:29:53 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): googlechromelabs.github.io:443
2025-02-16 16:29:53 [urllib3.connectionpool] DEBUG: https://googlechromelabs.github.io:443 "GET /chrome-for-testing/latest-patch-versions-per-build.json HTTP/1.1" 200 9720
2025-02-16 16:29:53 [WDM] INFO: Driver [/home/vjrdnti/.wdm/drivers/chromedriver/linux64/129.0.6668.100/chromedriver-linux64/THIRD_PARTY_NOTICES.chromedriver] found in cache
2025-02-16 16:29:53

Visiting The Independent: https://news.google.com/rss/articles/CBMi2AFBVV95cUxNem14cnE4M2REbjNIMEJCLVdJdkVxbFF5bXBLMWg5cFJDeW5HWlZMMjdoOEp2WThIWDAyUnlwVkVZbFZJcXVwdlVscUhoSDYxLU43OXFiTm1rREtoZ1JZcGppa1JQVlV4YU44MmZEWElCN1dUWFRwTGJWekhEdHptTDg1Y2F6NnVQY2pOZzFHUjNyUTkycm9IcEp0NUVuYlQ5NFBKVDBacHNSbmtCQWtzdmhrYVpTb0lEWk52ZXc3WkJqbk0tRlRoR1E4ZUlRUHRlZ1l0cmhQbG0?oc=5


2025-02-16 16:30:01 [urllib3.connectionpool] DEBUG: http://localhost:60323 "POST /session/551ecf1e1f60c794dd0c710df8feff9d/url HTTP/1.1" 200 14
2025-02-16 16:30:01 [selenium.webdriver.remote.remote_connection] DEBUG: Remote response: status=200 | data={"value":null} | headers=HTTPHeaderDict({'Content-Length': '14', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
2025-02-16 16:30:01 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2025-02-16 16:30:03 [selenium.webdriver.remote.remote_connection] DEBUG: GET http://localhost:60323/session/551ecf1e1f60c794dd0c710df8feff9d/url {}
2025-02-16 16:30:03 [urllib3.connectionpool] DEBUG: http://localhost:60323 "GET /session/551ecf1e1f60c794dd0c710df8feff9d/url HTTP/1.1" 200 153
2025-02-16 16:30:03 [selenium.webdriver.remote.remote_connection] DEBUG: Remote response: status=200 | data={"value":"https://www.independent.co.uk/news/world/americas/us-politics/trump-executive-orders-tariffs-hostage-d

Redirected to: https://www.independent.co.uk/news/world/americas/us-politics/trump-executive-orders-tariffs-hostage-doge-musk-live-news-b2698892.html?page=4


2025-02-16 16:30:04 [urllib3.connectionpool] DEBUG: http://localhost:60323 "POST /session/551ecf1e1f60c794dd0c710df8feff9d/url HTTP/1.1" 200 14
2025-02-16 16:30:04 [selenium.webdriver.remote.remote_connection] DEBUG: Remote response: status=200 | data={"value":null} | headers=HTTPHeaderDict({'Content-Length': '14', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
2025-02-16 16:30:04 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2025-02-16 16:30:07 [selenium.webdriver.remote.remote_connection] DEBUG: GET http://localhost:60323/session/551ecf1e1f60c794dd0c710df8feff9d/source {}
2025-02-16 16:30:07 [urllib3.connectionpool] DEBUG: http://localhost:60323 "GET /session/551ecf1e1f60c794dd0c710df8feff9d/source HTTP/1.1" 200 519893
2025-02-16 16:30:07 [selenium.webdriver.remote.remote_connection] DEBUG: Remote response: status=200 | data={"value":"\u003Chtml lang=\"en\">\u003Chead>\u003Cscript type=\"text/javascript\" async=\"\" src=\"http

2025-02-16 16:30:07 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2025-02-16 16:30:08 [selenium.webdriver.remote.remote_connection] DEBUG: DELETE http://localhost:60323/session/551ecf1e1f60c794dd0c710df8feff9d {}
2025-02-16 16:30:08 [urllib3.connectionpool] DEBUG: http://localhost:60323 "DELETE /session/551ecf1e1f60c794dd0c710df8feff9d HTTP/1.1" 200 14
2025-02-16 16:30:08 [selenium.webdriver.remote.remote_connection] DEBUG: Remote response: status=200 | data={"value":null} | headers=HTTPHeaderDict({'Content-Length': '14', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
2025-02-16 16:30:08 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request


In [64]:
scraped_results
#scraped result finna be cleaned and then combined and forwarded or directly forwarded for summarisation.
#content basic preprocessing can be handled easily but redundancies will need some work

[{'source': 'The Independent',
  'content': 'Notifications can be managed in browser preferences. Please refresh the page or navigate to another page on the site to be automatically logged inPlease refresh your browser to be logged in Trump shared a quote which is often attributed to the European conquerer over the weekend, while his administration fired 20 immigration judges without explanation Donald Trumpraised eyebrows on Saturday after he appeared to reference the dictatorial EmperorNapoleon Bonapartein an online post. “He who saves his Country does not violate any Law,” the president wrote onTruth Social, a quote which is often attributed to the European conquerer. It comes after several of Trump’s earlyexecutive ordersand budget-slashing withElon Musk\'s new Department of Government Efficiency have been called into court by a string of lawsuits. The pair are reportedly planningto accelerate theirpurge of America’s federal bureaucracy, with thousands of federal works laid off in 